In [5]:
import numpy as np
import pandas as pd
import os
import sys


parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from preprocess.preprocess import Preprocess
from regression_tree.regression_tree import MyDecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold

In [6]:
pp = Preprocess('support2.csv', 'death', scale=True)
results_df = pd.DataFrame(
)

In [7]:
X_train, X_test, y_train, y_test = pp.preprocess_data()

In [8]:
experiment_configs = [
    {
        "name": "Linear Regression",
        "model": Ridge(),
        "params": {} 
    },
    {
        "name": "Decision Tree",
        "model": DecisionTreeRegressor(random_state=42),
        "params": {"random_state": 42}
    },
    {
        "name": "Random Forest (10 Trees)",
        "model": RandomForestRegressor(random_state=42),
        "params": {"random_state": 42}
    },
    {
        "name": "MyDecisionTreeRegressor",
        "model": MyDecisionTreeRegressor(max_depth=20, min_samples=20),
        "params": {"pruned": True, "max_depth": 20, "min_samples": 20}
    },
    {
        "name": "MyDecisionTreeRegressor",
        "model": MyDecisionTreeRegressor(max_depth=15, min_samples=20),
        "params": {"pruned": False, "max_depth": 20, "min_samples": 20}
    }
]

results = []

# Loop for models
for config in experiment_configs:
    model = config["model"]
    name = config["name"]
    params = config["params"]
    
    # Train
    model.fit(X_train, y_train)
    
    if name == 'MyDecisionTreeRegressor':
        if config['params']['pruned']:
            model.post_prune_with_cross_validation(X_train, y_train, n_splits=5)

    
    # Predict
    y_pred = model.predict(X_test)
    
    # Evaluate
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Store results
    results.append({
        "model_name": name,
        "rsquared": r2,
        "rmse": rmse,
        "params": params
    })

# Create dataframe of the results
results_df = pd.DataFrame(results)

print(results_df)

                 model_name  rsquared      rmse  \
0         Linear Regression  0.542560  0.315635   
1             Decision Tree  0.586706  0.300018   
2  Random Forest (10 Trees)  0.764090  0.226668   
3   MyDecisionTreeRegressor  0.758801  0.229196   
4   MyDecisionTreeRegressor  0.694740  0.257842   

                                              params  
0                                                 {}  
1                               {'random_state': 42}  
2                               {'random_state': 42}  
3  {'pruned': True, 'max_depth': 20, 'min_samples...  
4  {'pruned': False, 'max_depth': 20, 'min_sample...  


In [9]:
import numpy as np
import pandas as pd
import copy
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone



cv = KFold(n_splits=10, shuffle=True, random_state=42)

results_cv = []


for config in experiment_configs:
    name = config["name"]
    base_model = config["model"]
    params = config["params"]
    
    
    fold_r2 = []
    fold_rmse = []
    
    for train_idx, val_idx in cv.split(X_train):
        
        # Create folds
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        if hasattr(base_model, 'get_params'):
             model = clone(base_model) # sklearn models
        else:
             model = copy.deepcopy(base_model) # custom models
        
        model.fit(X_fold_train, y_fold_train)
        model.fit(X_fold_train, y_fold_train)
        
        # Post-prune for the custom decision tree regressor
        if name == "MyDecisionTreeRegressor":
            if config['params']['pruned']:
                model.post_prune_with_cross_validation(X_fold_train, y_fold_train, n_splits=5)
        
        # Predictions
        y_pred = model.predict(X_fold_val)
        
        # Calculate metrics
        fold_r2.append(r2_score(y_fold_val, y_pred))
        fold_rmse.append(np.sqrt(mean_squared_error(y_fold_val, y_pred)))
    
    # Store results
    results_cv.append({
        "model_name": name,
        "mean_r2": np.mean(fold_r2),
        "mean_rmse": np.mean(fold_rmse)
    })

# Create datafram
df_results = pd.DataFrame(results_cv)
print(df_results)

                 model_name   mean_r2  mean_rmse
0         Linear Regression  0.560562   0.308445
1             Decision Tree  0.602794   0.292985
2  Random Forest (10 Trees)  0.796738   0.209774
3   MyDecisionTreeRegressor  0.785223   0.215438
4   MyDecisionTreeRegressor  0.720201   0.246127
